# the goal is to quickly see how we can convert to kmer

In [2]:
import itertools
import numpy as np
numbers = (7, 8, 9, 10, 11)
length = 6
dtype = np.int16 #can do int16 if length is 6, but otherwise int32.
#5^6 = 15625, 5^8 = 390625 so this needs int32

# Generate all combinations of length 8
combinations = itertools.product(numbers, repeat=length)

# Create a dictionary mapping each combination to a number
combination_dict = {comb: idx for idx, comb in enumerate(combinations)}

# Verify the dictionary size
print(f"Number of combinations: {len(combination_dict)}")  # Output: 15625 or 390625

from tqdm import tqdm

seqs = np.load('/data1/lesliec/sarthak/data/GraphReg/torchdata/seq.npy')

default_value = list(combination_dict.values())[-1]
newseqs = np.ones(seqs.shape, dtype=dtype)*default_value

print(seqs.shape)

Number of combinations: 15625
(1451, 6000000)


In [4]:
for j in range(seqs.shape[0]):
    for i in tqdm(range(seqs.shape[1]-length+1)):
        kmer = tuple(seqs[j][i:i+length])
        newseqs[j,i] = combination_dict[kmer]
#takes roughly 1 minute per thing, times 1451 is 1451 min

  0%|          | 0/5999995 [00:00<?, ?it/s]

 10%|█         | 618535/5999995 [00:05<00:50, 107611.80it/s]


KeyboardInterrupt: 

In [6]:
1451/60 #a full day to do it this way

24.183333333333334

In [1]:
# it's done now, let's test it
import numpy as np
seqs = np.load('/data1/lesliec/sarthak/data/GraphReg/torchdata/seq.npy')
s6 = np.load('/data1/lesliec/sarthak/data/GraphReg/torchdata/seqs_6mer.npy')
#print the shapes
print(seqs.shape, s6.shape)

(1451, 6000000) (1451, 6000000)


In [5]:
import itertools
numbers = (7, 8, 9, 10, 11)
length = 6

combinations = itertools.product(numbers, repeat=length)
print(seqs.nbytes/1e9, s6.nbytes/1e9)
combination_dict = {comb: idx for idx, comb in enumerate(combinations)}
print(len(combination_dict))

8.706 17.412
15625


In [6]:
combination_dict

{(7, 7, 7, 7, 7, 7): 0,
 (7, 7, 7, 7, 7, 8): 1,
 (7, 7, 7, 7, 7, 9): 2,
 (7, 7, 7, 7, 7, 10): 3,
 (7, 7, 7, 7, 7, 11): 4,
 (7, 7, 7, 7, 8, 7): 5,
 (7, 7, 7, 7, 8, 8): 6,
 (7, 7, 7, 7, 8, 9): 7,
 (7, 7, 7, 7, 8, 10): 8,
 (7, 7, 7, 7, 8, 11): 9,
 (7, 7, 7, 7, 9, 7): 10,
 (7, 7, 7, 7, 9, 8): 11,
 (7, 7, 7, 7, 9, 9): 12,
 (7, 7, 7, 7, 9, 10): 13,
 (7, 7, 7, 7, 9, 11): 14,
 (7, 7, 7, 7, 10, 7): 15,
 (7, 7, 7, 7, 10, 8): 16,
 (7, 7, 7, 7, 10, 9): 17,
 (7, 7, 7, 7, 10, 10): 18,
 (7, 7, 7, 7, 10, 11): 19,
 (7, 7, 7, 7, 11, 7): 20,
 (7, 7, 7, 7, 11, 8): 21,
 (7, 7, 7, 7, 11, 9): 22,
 (7, 7, 7, 7, 11, 10): 23,
 (7, 7, 7, 7, 11, 11): 24,
 (7, 7, 7, 8, 7, 7): 25,
 (7, 7, 7, 8, 7, 8): 26,
 (7, 7, 7, 8, 7, 9): 27,
 (7, 7, 7, 8, 7, 10): 28,
 (7, 7, 7, 8, 7, 11): 29,
 (7, 7, 7, 8, 8, 7): 30,
 (7, 7, 7, 8, 8, 8): 31,
 (7, 7, 7, 8, 8, 9): 32,
 (7, 7, 7, 8, 8, 10): 33,
 (7, 7, 7, 8, 8, 11): 34,
 (7, 7, 7, 8, 9, 7): 35,
 (7, 7, 7, 8, 9, 8): 36,
 (7, 7, 7, 8, 9, 9): 37,
 (7, 7, 7, 8, 9, 10): 38,
 (7, 7, 7,

In [7]:
#now let's take a random part of the sequence and see if it matches
test = seqs[832, 1000:1006]
print(test)
print(s6[832, 1000])
print(combination_dict[tuple(test)])

[11 11 11 11 11 11]
15624
15624


In [8]:
def test_kmer(seqs, s6, combination_dict, i,j, length=6):
    test = seqs[i, j:j+length]
    print(test)
    print(s6[i, j])
    print(combination_dict[tuple(test)])
    
test_kmer(seqs, s6, combination_dict, 832, 1000)

[11 11 11 11 11 11]
15624
15624


In [9]:
test_kmer(seqs, s6, combination_dict, 432, 100001)

[10  9 10  7  9  8]
11011
11011


In [10]:
s6[0,-100:] #last 5 are 15624, the default value which is fine

array([ 7278,  5143, 10093,  3591,  2330, 11650, 11377, 10012,  3186,
         305,  1527,  7638,  6942,  3461,  1682,  8412, 10811,  7181,
        4655,  7653,  7016,  3833,  3542,  2087, 10437,  5311, 10933,
        7792,  7712,  7312,  5311, 10931,  7783,  7667,  7087,  4186,
        5307, 10911,  7683,  7167,  4587,  7311,  5307, 10912,  7687,
        7187,  4686,  7806,  7783,  7667,  7086,  4183,  5292, 10838,
        7317,  5338, 11067,  8461, 11056,  8407, 10787,  7062,  4061,
        4682,  7787,  7687,  7186,  4680,  7777,  7635,  6928,  3391,
        1332,  6663,  2066, 10331,  4783,  8292, 10212,  4187,  5312,
       10936,  7808,  7792,  7710,  7301,  5256, 10658,  6417,   837,
        4185,  5300, 10877,  7511,  6308, 15624, 15624, 15624, 15624,
       15624], dtype=int16)

In [1]:
#and test the dataset
import src.dataloaders.datasets.graphreg_dataset as d
dataset = d.GraphRegDataset('test', 100_000, clean_data=True, remove_repeats=True, kmer_len=6)
a,b = dataset[0]
print(a.shape, b.shape)

base_path: /data1/lesliec/sarthak/
Using kmer genome with length 6
torch.Size([100000]) torch.Size([1000, 3])


In [5]:
#now load the other dataset
dataset_og = d.GraphRegDataset('test', 100_000, clean_data=True, remove_repeats=True)

In [16]:
import torch
import numpy as np
import itertools
numbers = (7, 8, 9, 10, 11)
length = 6

combinations = itertools.product(numbers, repeat=length)
combination_dict = {comb: idx for idx, comb in enumerate(combinations)}

for i in range(100):
    a1,b1 = dataset[i]
    a2,b2 = dataset_og[i]
    if not torch.allclose(b1, b2):
        print('issue')
    #now pick a random value between 0 and 90
    j = np.random.randint(0, 90_000)
    real = a1[j]
    compare = combination_dict[tuple(a2[j:j+6].numpy())]
    if real != compare:
        print('issue')

#I think it's safe to say it's good!!

In [13]:
#make sure the flip for rc aug works
import torch
test = torch.arange(3000).reshape(1000,3)
test

tensor([[   0,    1,    2],
        [   3,    4,    5],
        [   6,    7,    8],
        ...,
        [2991, 2992, 2993],
        [2994, 2995, 2996],
        [2997, 2998, 2999]])

In [15]:
test.flip(0) #reverses each column independently which is what we want!!

tensor([[2997, 2998, 2999],
        [2994, 2995, 2996],
        [2991, 2992, 2993],
        ...,
        [   6,    7,    8],
        [   3,    4,    5],
        [   0,    1,    2]])